<a href="https://colab.research.google.com/github/profteachkids/CHE2064_Spring2022/blob/main/Distillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/che.py
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/FlattenWrap.ipynb
!pip install importnb

In [2]:
from importnb import Notebook
with Notebook(): 
    from FlattenWrap import DotDict, Range, RangeArray, Comp, CompArray, dtox

import che
import numpy as np
from scipy.optimize import root

In [3]:
p=che.Props(['Benzene','Toluene'])

In [4]:
d=DotDict()

d.P = 150e3
d.N = 4
d.NC = p.Name.size
d.Feed = 100.
d.Feedz = np.array([0.5,0.5])
d.FT = 370.
d.D = d.Feed/2
d.B = d.Feed - d.D

d.F_stage = 2


d.U = np.zeros(d.N)
d.U[0]=d.D
d.U[-1]=d.B

d.reflux_ratio = 3.

# d.T = np.linspace(370,395,d.N-1)
# d.T = RangeArray(np.linspace(370, 395,d.N-1), 367., 398)
d.T = RangeArray([377.46245809, 382.48934105, 387.71326265], 367., 398.)
d.TC = Range(370., 360.,400.)
d.Lr = d.D * d.reflux_ratio
d.Ls = d.Lr + d.Feed 
d.L = RangeArray(np.concatenate([np.full(d.F_stage, d.Lr), np.full(d.N-d.F_stage-1, d.Ls)]), 0., 2*d.Feed*d.reflux_ratio)
d.Vr = d.D + d.Lr
d.Vs = d.Ls - d.B
d.V = RangeArray(np.concatenate([np.full(d.F_stage-1, d.Vr), np.full(d.N-d.F_stage, d.Vs)]), 0., 2*d.Feed*d.reflux_ratio)

QBGuess = d.D * d.reflux_ratio* np.mean(p.HvapNB)
d.QB = Range(QBGuess, 0., 2*QBGuess )


d.Lx = CompArray(np.array([[0.55902692, 0.44097308],
        [0.39606249, 0.60393751],
        [0.24765837, 0.75234163]]))
d.Vy =  CompArray(np.array([[0.75234163, 0.24765837],
        [0.6073556 , 0.3926444 ],
        [0.43316352, 0.56683648]]))

# d.Lx_ = CompArray(np.tile(d.Feedz, (d.N-1,1)))
# d.Vy_ = CompArray(np.tile(d.Feedz, (d.N-1,1)))

In [5]:
wrap, x, d2, xtod, xtodunk  = dtox(d)

In [6]:
r = DotDict()

r.MB = np.zeros((d.N-1,d.NC))
zeros = np.zeros((1,d.NC))

def eqs(d):
    r.EQUIL = d.Lx* p.Pvap(d.T) - d.Vy*d.P


    r.Lin = np.r_[(d.L[0]*d.Vy[0])[None,:], d.L[1:,None]*d.Lx[:-1]]

    r.Lout = np.r_[d.L[1:,None]*d.Lx[:-1], (d.B*d.Lx[-1])[None,:]]

    r.Vin = np.r_[d.V[1:,None]*d.Vy[1:], zeros]
    r.Vout = d.V[:,None]*d.Vy

    r.MB = r.Lin - r.Lout + r.Vin - r.Vout

    r.MB[d.F_stage-1] += d.Feed*d.Feedz

    r.TCBP = np.atleast_1d(np.sum(d.Vy[0]*p.Pvap(d.TC)) - d.P)

    r.T = np.insert(d.T,0,d.TC)
    r.EB = np.array(p.Hl(r.Lin, r.T[:-1]) + p.Hv(r.Vin, r.T[1:]) - p.Hl(r.Lout, r.T[1:]) - p.Hv(r.Vout, r.T[1:]))
    r.EB[d.F_stage-1] += np.array(p.Hl(d.Feed*d.Feedz, d.FT))
    r.EB[-1]+=d.QB

    r.RF = np.atleast_1d((d.V[0]-d.D)/d.D) - d.reflux_ratio
    return np.concatenate([np.ravel(r.EQUIL), np.ravel(r.MB), np.ravel(r.EB), r.TCBP, r.RF]), r


In [7]:
eqs_wrapped = wrap(eqs)

In [8]:
eqs(d2)

(array([-1.56644951e-03,  9.94893766e-04, -7.46569393e-04,  3.12923534e-04,
        -5.78236635e-04,  3.27505404e-07,  4.99999999e-07, -4.99999999e-07,
        -5.00000027e-07,  4.99999970e-07,  2.84217094e-14,  2.84217094e-14,
        -5.46036917e+04, -1.85654862e+05, -1.67519171e+06, -9.18765918e+03,
         0.00000000e+00]),
 {'EB': array([  -54603.69165614,  -185654.86166321, -1675191.70830553]),
  'EQUIL': DeviceArray([[-1.56644951e-03,  9.94893766e-04],
               [-7.46569393e-04,  3.12923534e-04],
               [-5.78236635e-04,  3.27505404e-07]], dtype=float64),
  'Lin': array([[112.8512445,  37.1487555],
         [ 83.854038 ,  66.145962 ],
         [ 99.0156225, 150.9843775]]),
  'Lout': array([[ 83.854038 ,  66.145962 ],
         [ 99.0156225, 150.9843775],
         [ 12.3829185,  37.6170815]]),
  'MB': array([[ 4.99999999e-07, -4.99999999e-07],
         [-5.00000027e-07,  4.99999970e-07],
         [ 2.84217094e-14,  2.84217094e-14]]),
  'RF': array([0.]),
  'T': arra

In [ ]:
sol=root(eqs_wrapped, x)
print(sol)
solx=sol.x

In [10]:
dunk=xtodunk(solx)
dunk

{'L': array([150.        , 149.98397173, 255.83763633]),
 'Lx': array([[0.55946779, 0.44053221],
        [0.39647357, 0.60352643],
        [0.2473177 , 0.7526823 ]]),
 'QB': array(6659741.070391),
 'T': array([377.44973285, 382.47578968, 387.72607076]),
 'TC': array(372.26289508),
 'V': array([200.        , 199.98397173, 205.83763633]),
 'Vy': array([[0.7526823 , 0.2473177 ],
        [0.60777529, 0.39222471],
        [0.43270501, 0.56729499]])}

In [11]:
eqs(xtod(sol.x,d2))

(array([ 1.25146471e-09,  3.20142135e-10,  1.05355866e-08, -1.49084372e-08,
        -8.00355338e-11, -9.45874490e-10, -3.38218342e-12, -3.41060513e-13,
        -1.86170723e-09,  4.61291449e-09,  1.86510363e-09, -4.61255922e-09,
         5.82432374e-04,  8.13282095e-05, -5.96297905e-05,  1.32130197e-04,
         0.00000000e+00]),
 {'EB': array([ 5.82432374e-04,  8.13282095e-05, -5.96297905e-05]),
  'EQUIL': DeviceArray([[ 1.25146471e-09,  3.20142135e-10],
               [ 1.05355866e-08, -1.49084372e-08],
               [-8.00355338e-11, -9.45874490e-10]], dtype=float64),
  'Lin': array([[112.90234546,  37.09765454],
         [ 83.91120182,  66.0727699 ],
         [101.43286046, 154.40477587]]),
  'Lout': array([[ 83.91120182,  66.0727699 ],
         [101.43286046, 154.40477587],
         [ 12.36588485,  37.63411515]]),
  'MB': array([[-3.38218342e-12, -3.41060513e-13],
         [-1.86170723e-09,  4.61291449e-09],
         [ 1.86510363e-09, -4.61255922e-09]]),
  'RF': array([0.]),
  'T'